In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D
from keras.layers import Merge,Input
#from keras.layers.merge import Concatenate
wnl = WordNetLemmatizer()
stopwords = stopwords.words('english')

Using TensorFlow backend.


In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv',sep='\t',encoding='utf8')

In [3]:
import tensorflow


In [4]:
def clean_data(string):
    try: 
        rx = re.compile('\W+')
        res = rx.sub(' ',string).strip()
        new_str = ''
        #print(res)
        for word in res.split():
            word = wnl.lemmatize(word).lower() if word not in stopwords else ''
            new_str+=word + ' '
        return new_str
    except TypeError:
        print('TypeError',string)

In [5]:
df.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
df['q1_clean'] = df['question1'].apply(lambda x: clean_data(x))
df['q2_clean'] = df['question2'].apply(lambda x: clean_data(x))

TypeError nan
TypeError nan


In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_clean,q2_clean
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,what step step guide invest share market ...,what step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,what story kohinoor koh noor diamond,what would happen indian government stole k...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,how i increase speed internet connection ...,how internet speed increased hacking dns
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,why i mentally lonely how i solve,find remainder math 23 24 math divided 24 23
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,which one dissolve water quikly sugar salt me...,which fish would survive salt water


In [8]:
def build_vocab(string):
    for x in string.split():
        vocab.add(x)
        lexicon.append(x)
    return

In [9]:
vocab = set()
lexicon = []

In [10]:
df.dropna().shape

(404288, 8)

In [11]:
df = df.dropna()

In [12]:
_ = df['q1_clean'].apply(lambda x: build_vocab(x))
_ = df['q2_clean'].apply(lambda x: build_vocab(x))

In [13]:
vocab_size = len(vocab)

In [14]:
def generate_labels(inputs, cnter,max_len,vocab_len):
    np_input = []
    
    for sentences in inputs:
        sen  = np.full(max_len,vocab_len)
        for idx,word in enumerate(sentences.split()):
            if(idx<max_len):
                sen[idx] = cnter.index(word)
        np_input.append(sen)
        
    return np.array(np_input).astype(np.int)
        
        

In [15]:
from collections import Counter
vocab_cnter = Counter(lexicon)

In [16]:
sorted_vocab = [x[0] for x in vocab_cnter.most_common()]

In [17]:
q1_inputs = generate_labels(df['q1_clean'],sorted_vocab,100,len(vocab))
print('q1 processed')
q2_inputs = generate_labels(df['q2_clean'],sorted_vocab,100,len(vocab))

q1 processed


In [18]:
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate,Input, concatenate
#from keras.layers.merge import Concatenate
embedding_dim = 64
sentence_len = 100
# Q1
input_1 = Input(shape=(100,))
embedding_1 = Embedding(len(vocab),64, name="Embedding_1")(input_1)
bi_gru_1 = Bidirectional(GRU(128,name='GRU_1'))(embedding_1)

#Q2 
input_2 = Input(shape=(100,))
embedding_2 = Embedding(len(vocab),64,name="Embedding_2")(input_2)
bi_gru_2 = Bidirectional(GRU(128,name="GRU_2"))(embedding_2)
#Merge
#concat = Concatenate(axis=1)
merged = concatenate([bi_gru_1,bi_gru_2])
#merged = (Flatten())(merged)
#Dense
dense = Dense(1000,activation='relu',name='dense')(merged)
pred = Dense(2,activation="softmax",name='pred')(dense)
model = Model(inputs=[input_1,input_2],outputs=pred)
# model = Sequential()
# model.add(Embedding(len(vocab),64,input_length=sentence_len,name='Embedding_Input'))
# model.add(Bidirectional(GRU(100,dropout=.5,name='GRU')))
# model.add(Dense(1000,activation='relu',))

In [21]:
q1_inputs.shape

(404288, 100)

In [22]:
q2_inputs.shape

(404288, 100)

In [27]:
df['is_duplicate'].values.shape

(404288,)

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split([q1_inputs,q2_inputs],df['is_duplicate'].values,test_size =.2,random_state=4)

ValueError: Found input variables with inconsistent numbers of samples: [2, 404288]

In [32]:
pd.get_dummies(df['is_duplicate']).values

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [ ]:
from keras import optimizers
adam = optimizers.Adam()
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(x=[q1_inputs,q2_inputs],y=pd.get_dummies(df['is_duplicate']).values,batch_size=32,epochs=20,validation_split=.1)

Train on 363859 samples, validate on 40429 samples
Epoch 1/20
 32896/363859 [=>............................] - ETA: 2582s - loss: 0.5798 - acc: 0.7023

In [ ]:
def embedding_layer(scope_name,vocab_size,embedding_dim):
    w = tf.Variable(tf.random_uniform([vocab_size,embedding_dim],-1.0,1.0),name=scope_name)
    
    

In [34]:
df.is_duplicate.value_counts()

0    255025
1    149263
Name: is_duplicate, dtype: int64

In [37]:
def load_glove(filename):
    print('loading glove')
    f = open(filename,'r',encoding='utf8')
    model = {}
    for line in f:
        splitline = line.split()
        word = splitline[0]
        embedding = [float(val) for val in splitline[1:]]
        model[word] = embedding
    print("Done",len(model),'words loaded')
    return model

In [ ]:
glove = load_glove('glove.42B.300d.txt')

loading glove


In [ ]:
with open('GoogleNews-vectors-negative300.bin',rb) as f:
    header = f.readline()
    vocab_size, layer1_size = map(int,header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in xrange(vocab_size):
        word = []
        while True:
            ch = f.read(1)
            if ch == ' ':
                word = ''.join(word)
                break
            if ch !='\n':
                word.append(ch)
            idx = vocab_processor.vocabulary_.get(word)
            if idx !=0:
                initW[idx] = np.fromstring(f.read(binary_len),dtype='float32')
            else:
                f.read(binary_len)
            
            
       